This notebook shows how to take an average over an ensemble of neural networks. This does not actually work particularly well, but serves as a useful proof of concept.

In [ ]:
import glob
from lib.plots.single_column_initial_value import column_run
from pathlib import Path
import torch
from torch import nn 
from functools import reduce

from lib.models.torch.multiple_step_objective import ForcedStepper


def _sum_rhs(x, y, n=1):
    return [
        {key: xx[key] + yy[key] for key in xx}
        for xx, yy in zip(x, y)
    ]

def _normalize(x, n):
    return [
        {key: xx[key]/n for key in xx}
        for xx in x
    ]

class Mean(nn.ModuleList):        
    def forward(self, *args, **kwargs):
        s =  reduce(_sum_rhs, (mod(*args, **kwargs) for mod in self))
        return _normalize(s, len(self))

        

def mean_model(models):
    rhs = Mean([model.rhs for model in models])
    
    stepper = models[0]
    stepper = ForcedStepper(rhs, stepper.h, stepper.nsteps,
                            interactive_vertical_adv=stepper.interactive_vertical_adv)
    
    return stepper
    



In [ ]:
inputs = xr.open_dataset("../data/processed/inputs.nc").stack(batch=['x', 'y'])
forcings = xr.open_dataset("../data/processed/forcings.nc").stack(batch=['x', 'y'])

In [ ]:
model_id = Path("../data/output/model.VaryNHid-128")
models = [torch.load(f) for f in glob.glob(str(model_id / "*.torch"))]
stepper = mean_model(models)

data = column_run(stepper, inputs, forcings)

In [ ]:
%matplotlib inline

In [ ]:
ds = xr.Dataset(data[0]).unstack('batch')
tru = inputs.unstack('batch')

In [ ]:
bias = (ds-tru).mean(['x', 'time'])

In [ ]:
bias.qt.isel(y=8).plot()

In [ ]:
bias.sl.isel(y=8).plot()

We see that 